In [1]:

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_distances
from sklearn.neighbors import NearestNeighbors
import umap
import matplotlib.pyplot as plt

df_full = pd.read_csv("../Adata/yahoo/train_subset.csv").reset_index(drop=True)
texts_full = df_full["text"].tolist()
labels_full = df_full["label"].tolist()

/home/julita/anaconda3/envs/anote/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts_full) 

In [3]:
import numpy as np

# Przygotowanie sąsiedztw do pokrycia: dla każdego punktu określ listę innych próbek, które by pokrył 
# (np. wszystkie w odległości kosinusowej > delta)
delta = 0.8
N = len(embeddings)
neighbors = [set() for _ in range(N)]
for i in range(N):
    for j in range(N):
        if i == j:
            continue
        if np.dot(embeddings[i], embeddings[j]) > delta:
            neighbors[i].add(j)

B = 50  # budżet
selected = []
covered = set()
while len(selected) < B:
    best_gain = 0
    best_idx = None
    # znajdź punkt dający największy przyrost pokrycia
    for i in range(N):
        if i in selected:
            continue
        # policz ile nowych punktów pokryje dodanie i
        gain = len(neighbors[i] - covered)
        if gain > best_gain:
            best_gain = gain
            best_idx = i
    if best_idx is None:
        break
    # dodaj best_idx do wybranych i zaktualizuj zbiór pokrytych
    selected.append(best_idx)
    covered |= neighbors[best_idx]

# 'selected' zawiera indeksy wybranych próbek maksymalizujących pokrycie (dla zadanego delta).


In [4]:
selected

[114, 933]